# Data Import from NCBI GenBank


In [32]:
from Bio import SeqIO
from Bio import Entrez
Entrez.email = "jhwnkim@mit.edu"  # Always tell NCBI who you are

import traceback

# Read Covid19 reference sequence
ref = SeqIO.read("./data/ref_sequence.gb", "genbank")
print('Reference Covid sequence')
print(ref.id)
print(repr(ref.seq))
print(len(ref.seq))


def get_meta(record_id='NC_045512.2'): # default is the reference
    handle = Entrez.efetch(db="nuccore", id=record_id, rettype="gb", retmode="xml")
    fetch_record = Entrez.read(handle)    
    handle.close()
    
    metadata = {
        'id': record_id,
        'submit-date': fetch_record[0]['GBSeq_create-date'],
    }
    
    for qualifier in fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals']:
        if qualifier['GBQualifier_name'] == 'country':
            metadata['country'] = qualifier['GBQualifier_value']
        if qualifier['GBQualifier_name'] == 'collection_date':
            metadata['collect-date'] = qualifier['GBQualifier_value']

    print(metadata)
    return metadata


from Bio import Align
from Bio.Align.substitution_matrices import Array

def mutation_array(seq1, seq2):
    aligner = Align.PairwiseAligner()
    aligner.mode = 'local'
    aligner.match_score = 2
    aligner.mismatch_score = -3
    aligner.open_gap_score = -7
    aligner.extend_gap_score = -2

    alignments = aligner.align(seq1, seq2)
    alignment = alignments[0]    
    
    frequency = Array("ACGTN", dims=2)
    for (start1, end1), (start2, end2) in zip(*alignment.aligned):
        se1 = seq1[start1:end1]
        se2 = seq2[start2:end2]
        for c1, c2 in zip(se1, se2):
            frequency[c1, c2] += 1
            
    print(frequency)
    mut_rate = frequency / len(seq1)
    print(mut_rate)
    
    return mut_rate

# Read downloaded sequence file from NCBI GenBank Virus site
# records = list( SeqIO.parse("./data/MA-sequences.fasta", "fasta") )
# records = list( SeqIO.parse("./data/MA-sequences-1-toy.fasta", "fasta") )
records = list( SeqIO.parse("./data/MA-sequences-1.fasta", "fasta") )

metadata = []
mutarray = []

for idx, record in enumerate(records):
    print('\n{} of {} records'.format(idx+1, len(records)))
    try:
        meta = get_meta(record.id)
        mut = mutation_array(ref.seq, record.seq)
    except:
        print(traceback.format_exc())
    else:    
        metadata.append(meta)
        mutarray.append(mut)
    


Reference Covid sequence
NC_045512.2
Seq('ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGATCTCTTGT...AAA')
29903
{'id': 'MZ149328.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-29'}
       A      C      G      T   N
A 8934.0    0.0    3.0    3.0 0.0
C    4.0 5463.0    0.0   17.0 0.0
G    4.0    4.0 5842.0    6.0 0.0
T    1.0    2.0    1.0 9571.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149330.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-28'}
       A      C      G      T    N
A 8932.0    0.0    3.0    3.0  2.0
C    3.0 5463.0    0.0   15.0  3.0
G    4.0    3.0 5842.0    4.0  3.0
T    1.0    2.0    1.0 9556.0 15.0
N    0.0    0.0    0.0    0.0  0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N

{'id': 'MZ149376.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-05-01'}
       A      C      G      T   N
A 8934.0    0.0    2.0    4.0 0.0
C    3.0 5466.0    0.0   15.0 0.0
G    2.0    4.0 5849.0    1.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149378.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-28'}
       A      C      G      T   N
A 8931.0    1.0    6.0    0.0 0.0
C    1.0 5466.0    0.0   12.0 0.0
G    2.0    1.0 5845.0    1.0 0.0
T    0.0    1.0    1.0 9573.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149379.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-d

       A      C      G      T   N
A 8936.0    0.0    2.0    3.0 0.0
C    3.0 5466.0    0.0   15.0 0.0
G    3.0    4.0 5848.0    1.0 0.0
T    1.0    2.0    1.0 9571.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149423.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-28'}
       A      C      G      T   N
A 8924.0    0.0    2.0    2.0 5.0
C    3.0 5453.0    0.0   17.0 3.0
G    3.0    3.0 5843.0    1.0 4.0
T    1.0    2.0    1.0 9558.0 5.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149425.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-30'}
       A      C      G      T   N
A 8935.0    0.0    2.0    3.0 0.0
C    3.0 5465.0    0.0   1

{'id': 'MZ149462.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-29'}
       A      C      G      T   N
A 8928.0    0.0    3.0    2.0 0.0
C    5.0 5458.0    0.0   13.0 0.0
G    5.0    3.0 5844.0    2.0 0.0
T    1.0    4.0    1.0 9561.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149465.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-30'}
       A      C      G      T   N
A 8936.0    0.0    3.0    2.0 0.0
C    3.0 5463.0    0.0   18.0 0.0
G    2.0    3.0 5848.0    3.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149466.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-d

       A      C      G      T   N
A 8936.0    0.0    2.0    3.0 0.0
C    3.0 5467.0    0.0   14.0 0.0
G    2.0    3.0 5850.0    1.0 0.0
T    2.0    6.0    2.0 9565.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149515.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-05-01'}
       A      C      G      T   N
A 8932.0    0.0    6.0    2.0 0.0
C    3.0 5468.0    0.0   13.0 0.0
G    2.0    3.0 5849.0    2.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ149522.1', 'submit-date': '09-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-27'}
       A      C      G      T   N
A 8932.0    0.0    6.0    2.0 0.0
C    3.0 5467.0    0.0   1

...

       A      C      G      T   N
A 8936.0    0.0    3.0    2.0 0.0
C    3.0 5469.0    0.0   12.0 0.0
G    2.0    3.0 5847.0    4.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145466.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-26'}
       A      C      G      T   N
A 8923.0    1.0    6.0    1.0 0.0
C    0.0 5458.0    0.0   13.0 0.0
G    2.0    0.0 5844.0    1.0 0.0
T    0.0    2.0    1.0 9564.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145470.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-27'}
       A      C      G      T   N
A 8935.0    0.0    2.0    3.0 0.0
C    3.0 5461.0    0.0   2

{'id': 'MZ145515.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-27'}
       A      C      G      T   N
A 8935.0    0.0    2.0    3.0 0.0
C    3.0 5463.0    0.0   18.0 0.0
G    3.0    3.0 5848.0    2.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145516.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-26'}
       A      C      G      T   N
A 8930.0    1.0    8.0    0.0 0.0
C    0.0 5467.0    0.0   12.0 0.0
G    2.0    0.0 5846.0    1.0 0.0
T    0.0    1.0    1.0 9573.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145517.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-d

       A      C      G      T   N
A 8926.0    1.0    8.0    0.0 3.0
C    0.0 5462.0    0.0   17.0 0.0
G    2.0    0.0 5843.0    2.0 2.0
T    0.0    3.0    1.0 9568.0 3.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145551.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-27'}
       A      C      G      T   N
A 8935.0    0.0    3.0    3.0 0.0
C    3.0 5464.0    0.0   17.0 0.0
G    2.0    3.0 5849.0    2.0 0.0
T    1.0    3.0    1.0 9570.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145552.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-27'}
       A      C      G      T   N
A 8936.0    0.0    3.0    2.0 0.0
C    3.0 5470.0    0.0   1

{'id': 'MZ145582.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-26'}
       A      C      G      T   N
A 8935.0    0.0    3.0    2.0 0.0
C    3.0 5467.0    0.0   14.0 0.0
G    2.0    3.0 5849.0    2.0 0.0
T    1.0    2.0    1.0 9571.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145583.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-date': '2021-04-26'}
       A      C      G      T   N
A 8939.0    1.0    3.0    0.0 0.0
C    0.0 5471.0    0.0   14.0 0.0
G    2.0    1.0 5851.0    3.0 0.0
T    0.0    1.0    0.0 9578.0 0.0
N    0.0    0.0    0.0    0.0 0.0

    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0


{'id': 'MZ145587.1', 'submit-date': '07-MAY-2021', 'country': 'USA: Massachusetts', 'collect-d

IncompleteRead: IncompleteRead(14 bytes read)

In [3]:
from Bio import SeqIO
records = list( SeqIO.parse("./data/MA-sequences-1-toy-add.fasta", "fasta") )

# print(records)
print(records[0].description)

import re
match = re.search(r'(\d+-\d+-\d+)',records[0].description)

match.group(1)


MZ149328.1 |Severe acute respiratory syndrome coronavirus 2 isolate SARS-CoV-2/human/USA/MA-CDCBI-CRSP_26JQOUM65E5MBC6X/2021, complete genome|USA: Massachusetts|2021-04-29|MA


'2021-04-29'

NameError: name 'records' is not defined

In [34]:
dates = []
mutarray_avg = []
ids = []

for idx, rec in enumerate(metadata):
    if len(mutarray_avg) ==0 or rec['collect-date'] > dates[-1]:
        dates.append(rec['collect-date'])
        ids.append([rec['id']])
        mutarray_avg.append(mutarray[idx])
        
    else:
        for i in range(len(mutarray_avg)):
            if rec['collect-date']< dates[i]:
                dates.insert(i, rec['collect-date'])
                ids.insert(i,[rec['id']])
                mutarray_avg.insert(i, mutarray[idx])
                break
            elif rec['collect-date'] == dates[i]:
                ids[i].append(rec['id'])
                mutarray_avg[i] += mutarray[idx]
                break
                
# Divide mutation rate by counts
for idx, idlist in enumerate(ids):
    mutarray_avg[idx] = mutarray_avg[idx]/len(idlist)
    
    print(dates[idx])
    print(idlist)
    print(mutarray_avg[idx])
    
    
# print('{:e}'.format(mutarray_avg[0]['C', 'T']))
# Save to file
import pickle

outfile = './data/MA-sequences-1.dat'
data = {
    'dates': dates,
    'idlist': idlist,
    'mutarray_avg': mutarray_avg
}
with open(outfile, "wb") as f:
    pickle.dump(data, f)
# with open(outfile, "rb") as f:
#     print pickle.load(f)

2021-04-26
['MZ145403.1', 'MZ145406.1', 'MZ145407.1', 'MZ145410.1', 'MZ145415.1', 'MZ145419.1', 'MZ145421.1', 'MZ145428.1', 'MZ145431.1', 'MZ145437.1', 'MZ145439.1', 'MZ145445.1', 'MZ145447.1', 'MZ145448.1', 'MZ145454.1', 'MZ145455.1', 'MZ145457.1', 'MZ145458.1', 'MZ145466.1', 'MZ145478.1', 'MZ145480.1', 'MZ145483.1', 'MZ145487.1', 'MZ145491.1', 'MZ145494.1', 'MZ145496.1', 'MZ145498.1', 'MZ145506.1', 'MZ145507.1', 'MZ145508.1', 'MZ145516.1', 'MZ145517.1', 'MZ145519.1', 'MZ145521.1', 'MZ145525.1', 'MZ145536.1', 'MZ145537.1', 'MZ145539.1', 'MZ145547.1', 'MZ145553.1', 'MZ145555.1', 'MZ145558.1', 'MZ145561.1', 'MZ145564.1', 'MZ145567.1', 'MZ145568.1', 'MZ145570.1', 'MZ145574.1', 'MZ145582.1', 'MZ145583.1', 'MZ145588.1', 'MZ145595.1', 'MZ145597.1', 'MZ145600.1', 'MZ145613.1']
    A   C   G   T   N
A 0.3 0.0 0.0 0.0 0.0
C 0.0 0.2 0.0 0.0 0.0
G 0.0 0.0 0.2 0.0 0.0
T 0.0 0.0 0.0 0.3 0.0
N 0.0 0.0 0.0 0.0 0.0

2021-04-27
['MZ149387.1', 'MZ149435.1', 'MZ149452.1', 'MZ149474.1', 'MZ149522.1', 'MZ

In [35]:
# Save to file
import pickle

outfile = './data/MA-sequences-1.dat'
data = {
    'dates': dates,
    'idlist': idlist,
    'mutarray_avg': mutarray_avg
}
with open(outfile, "wb") as f:
    pickle.dump(data, f)

In [14]:
# examine record

import json

from Bio import Entrez
Entrez.email = "jhwnkim@mit.edu"  # Always tell NCBI who you are


handle = Entrez.efetch(db="nuccore", id='NC_045512.2', rettype="gb", retmode="xml")
fetch_record = Entrez.read(handle)    
handle.close()
# fetch_record[0].keys()
print(fetch_record[0]['GBSeq_create-date'])
# print(json.dumps(fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals'], indent=1))
# print(json.dumps(fetch_record[0]['GBSeq_feature-table'][0], indent=1))
print(json.dumps(fetch_record[0], indent=1))

print()


for qualifier in fetch_record[0]['GBSeq_feature-table'][0]['GBFeature_quals']:
    if qualifier['GBQualifier_name'] == 'country':
        print(qualifier['GBQualifier_value'] )
# print(json.dumps(fetch_record[0], indent=1))

13-JAN-2020
{
 "GBSeq_locus": "NC_045512",
 "GBSeq_length": "29903",
 "GBSeq_strandedness": "single",
 "GBSeq_moltype": "RNA",
 "GBSeq_topology": "linear",
 "GBSeq_division": "VRL",
 "GBSeq_update-date": "18-JUL-2020",
 "GBSeq_create-date": "13-JAN-2020",
 "GBSeq_definition": "Severe acute respiratory syndrome coronavirus 2 isolate Wuhan-Hu-1, complete genome",
 "GBSeq_primary-accession": "NC_045512",
 "GBSeq_accession-version": "NC_045512.2",
 "GBSeq_other-seqids": [
  "ref|NC_045512.2|",
  "gi|1798174254"
 ],
 "GBSeq_project": "PRJNA485481",
 "GBSeq_keywords": [
  "RefSeq"
 ],
 "GBSeq_source": "Severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2)",
 "GBSeq_organism": "Severe acute respiratory syndrome coronavirus 2",
 "GBSeq_taxonomy": "Viruses; Riboviria; Orthornavirae; Pisuviricota; Pisoniviricetes; Nidovirales; Cornidovirineae; Coronaviridae; Orthocoronavirinae; Betacoronavirus; Sarbecovirus",
 "GBSeq_references": [
  {
   "GBReference_reference": "1",
   "GBReference_posi

In [21]:
print('2021-04-29' < '2021-12-01')
print('2021-04-29' > '2021-04-30')
print('2021-04-29' == '2021-04-30')

True
False
False


In [ ]:
# Test alignment
# Import pairwise2 module
from Bio import pairwise2


# Import format_alignment method
from Bio.pairwise2 import format_alignment

# Define two sequences to be aligned
X = ref.seq
Y = records[0].seq

# Get a list of the global alignments between the two sequences ACGGGT and ACG satisfying the given scoring
# A match score is the score of identical chars, else mismatch score.
# Same open and extend gap penalties for both sequences.
alignments = pairwise2.align.globalms(X, Y, 2, -1, -0.5, -0.1)

# # Use format_alignment method to format the alignments in the list
# for a in alignments:
#     print(format_alignment(*a))

In [16]:
from Bio import Align
aligner = Align.PairwiseAligner()
aligner.mode = 'local'
aligner.match_score = 2
aligner.mismatch_score = -3
aligner.open_gap_score = -7
aligner.extend_gap_score = -2

# seq1 = "GAACT"
# seq2 = "GAT"

seq1 = ref.seq
seq2 = records[0].seq

# score = aligner.score(seq1, seq2)

alignments = aligner.align(seq1, seq2)
# print(len(alignments))
# for alignment in alignments:
#     print(alignment.score)
#     print(alignment)
alignment = alignments[0]
from Bio.Align.substitution_matrices import Array
frequency = Array("ACGT", dims=2)
for (start1, end1), (start2, end2) in zip(*alignment.aligned):
    se1 = seq1[start1:end1]
    se2 = seq2[start2:end2]
    for c1, c2 in zip(se1, se2):
        frequency[c1, c2] += 1
print(frequency[0,:])
print(alignment.score)



       A      C      G      T
A 8901.0    1.0    3.0    0.0
C    1.0 5469.0    0.0   14.0
G    2.0    0.0 5852.0    1.0
T    0.0    1.0    0.0 9577.0

59499.0


In [17]:
print(frequency[0,:])

A 8901.0
C    1.0
G    3.0
T    0.0



In [12]:
alignment = alignments[2]
from Bio.Align.substitution_matrices import Array
frequency = Array("ACGT", dims=2)
for (start1, end1), (start2, end2) in zip(*alignment.aligned):
    se1 = seq1[start1:end1]
    se2 = seq2[start2:end2]
    for c1, c2 in zip(se1, se2):
        frequency[c1, c2] += 1
print(frequency)
print(alignment.score)

       A      C      G      T
A 8901.0    1.0    3.0    0.0
C    1.0 5469.0    0.0   14.0
G    2.0    0.0 5852.0    1.0
T    0.0    1.0    0.0 9577.0

59499.0
